In [9]:
# Import the required libraries
import pandas as pd
import networkx as nx

In [10]:
edges_df = pd.read_csv('../data/gephi-edge-list.csv')
nodes_df = pd.read_csv('../data/gephi-node-list.csv')

In [11]:
# Create a mapping from ID to name
id_to_name = dict(zip(nodes_df['ID'], nodes_df['name']))

# Initialize graph
G = nx.Graph()

# Add nodes with names as attributes
for _, row in nodes_df.iterrows():
    G.add_node(row['ID'], name=row['name'])

# Add edges with weights
for _, row in edges_df.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['Weight'])

In [12]:
# Compute sum of neighbor edge weights (and use names in output)
sum_neighbor_weights = {}

for node_id in G.nodes():
    neighbors = list(G.neighbors(node_id))
    seen_edges = set()  # To avoid double-counting undirected edges
    total = 0
    
    for neighbor in neighbors:
        for nbr_of_neighbor in G.neighbors(neighbor):
            edge = tuple(sorted([neighbor, nbr_of_neighbor]))
            if edge not in seen_edges:
                total += G[neighbor][nbr_of_neighbor]['weight']
                seen_edges.add(edge)
    
    node_name = G.nodes[node_id]['name']
    sum_neighbor_weights[node_name] = total

# Convert to DataFrame
df = pd.DataFrame(sum_neighbor_weights.items(), columns=["Node", "Sum of Neighbor's Edge Weights"])
print(df)

                Node  Sum of Neighbor's Edge Weights
0     Adam Vinatieri                    48262.125519
1       Jason Peters                        0.000000
2     Terence Newman                    24527.738000
3      DeAngelo Hall                     6889.435667
4       Robbie Gould                     8893.807460
...              ...                             ...
1951   Michael Penix                    75897.802000
1952  Caleb Williams                    95249.109286
1953     Rome Odunze                    95249.109286
1954  Malachi Corley                   101174.800154
1955     Trey Benson                    71622.545000

[1956 rows x 2 columns]
